<a href="https://colab.research.google.com/github/Leads-DigiSaka-System/digisaka_gpt/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import random

# --------------------------
# Config
# --------------------------
num_payload_examples = 1000   # Number of payload examples
num_response_examples = 20    # Number of API response summarization examples
output_file = "generated_dataset.json"

# --------------------------
# Helper functions
# --------------------------
def random_image_id():
    """Generate a random image ID like img12345.png"""
    return f"img{random.randint(10000, 99999)}.png"

def create_payload_example(tool):
    img_id = random_image_id()
    if tool == "tree_detection":
        instruction = "Convert user request into API payload JSON only."
        user_input = f"Detect trees in image {img_id}."
        output = json.dumps({"tool": "tree_detection", "params": {"image_id": img_id}})
    elif tool == "crown_segmentation":
        instruction = "Convert user request into API payload JSON only."
        user_input = f"Segment tree crowns in image {img_id}."
        output = json.dumps({"tool": "crown_segmentation", "params": {"image_id": img_id}})
    else:
        raise ValueError("Unknown tool")
    return {"instruction": instruction, "input": user_input, "output": output}

def create_response_example(tool, detections, segments=None):
    """Create API response summarization example"""
    if tool == "tree_detection":
        instruction = "Convert API JSON result into human readable answer."
        api_input = json.dumps({"success": True, "total_detections": detections})
        output = f"The tree detection tool successfully detected {detections} trees in the uploaded image."
    elif tool == "crown_segmentation":
        instruction = "Convert API JSON result into human readable answer."
        api_input = json.dumps({"success": True, "total_detections": detections, "total_segments": segments})
        output = f"The crown segmentation tool successfully detected {detections} trees and segmented {segments} crowns in the uploaded image."
    else:
        raise ValueError("Unknown tool")
    return {"instruction": instruction, "input": api_input, "output": output}

# --------------------------
# Generate dataset
# --------------------------
dataset = []

# Generate payload examples (randomly alternate tools)
for _ in range(num_payload_examples):
    tool = random.choice(["tree_detection", "crown_segmentation"])
    dataset.append(create_payload_example(tool))

# Generate API response examples
for _ in range(num_response_examples):
    detections = random.randint(100, 300)
    segments = detections - random.randint(0, 5)  # Slightly fewer segments than trees
    tool = random.choice(["tree_detection", "crown_segmentation"])
    if tool == "tree_detection":
        dataset.append(create_response_example(tool, detections))
    else:
        dataset.append(create_response_example(tool, detections, segments))

# --------------------------
# Save to JSON
# --------------------------
with open(output_file, "w") as f:
    json.dump(dataset, f, indent=2)

print(f"Dataset generated: {len(dataset)} examples -> {output_file}")


Dataset generated: 1020 examples -> generated_dataset.json


In [ ]:
# Load geospatial dataset
# First run: !python generate_geo_dataset.py (or upload generate_geo_dataset.py and run it)
# Then load the generated dataset here

import json

try:
    file = json.load(open("/content/generated_dataset.json", "r"))
    print(f"✓ Loaded {len(file)} geospatial examples")
    print(f"\nSample example:")
    print(f"Q: {file[0]['input']}")
    print(f"A: {file[0]['output'][:150]}...")
except FileNotFoundError:
    print("⚠ Dataset not found!")
    print("Please run: !python generate_geo_dataset.py")
    print("Or upload generate_geo_dataset.py and execute it first.")
    file = []

✓ Loaded 1020 geospatial examples

Sample example:
Q: Detect trees in image img14156.png.
A: {"tool": "tree_detection", "params": {"image_id": "img14156.png"}}...


In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2026.1.4: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
from datasets import Dataset
import json

def format_prompt(example):
    """Format dataset examples for training - handles both formats"""
    # Check if example has instruction field (datasets_llm.json format)
    if 'instruction' in example:
        instruction = example.get('instruction', '')
        input_text = example.get('input', '')
        output_text = example.get('output', '')
        # Format with instruction
        return f"### Instruction: {instruction}\n### Input: {input_text}\n### Output: {output_text}<|endoftext|>"
    else:
        # Old format (just input/output)
        return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

print(f"✓ Formatted {len(formatted_data)} examples for training")
print(f"\nSample formatted example:")
print(formatted_data[0][:300] + "..." if len(formatted_data[0]) > 300 else formatted_data[0])

✓ Formatted 1020 examples for training

Sample formatted example:
### Instruction: Convert user request into API payload JSON only.
### Input: Detect trees in image img14156.png.
### Output: {"tool": "tree_detection", "params": {"image_id": "img14156.png"}}<|endoftext|>


In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1020 [00:00<?, ? examples/s]

In [ ]:
from unsloth import FastLanguageModel

FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "system",
        "content": "Convert user request into API payload JSON only. Output ONLY valid JSON."
    },
    {
        "role": "user",
        "content": "Please segment the tree crowns in drone_image_88921.png"
    }
]


inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

# ✅ inputs is already a Tensor
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=120,
    do_sample=False,
    temperature=0.0,
    top_p=1.0,
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract JSON only
start = response.find("{")
end = response.rfind("}") + 1
json_output = response[start:end]

print("JSON Output:\n", json_output)


JSON Output:
 {"tool": "crown_segmentation", "params": {"image_id": "drone_image_88921.png"}}


In [ ]:
messages = [
    {
        "role": "system",
        "content": "Convert API JSON result into human readable answer."
    },
    {
        "role": "user",
        "content": "{\"success\": true, \"total_detections\": 120, \"total_segments\": 118}"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=100,
    do_sample=False,
    temperature=0.0,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Convert API JSON result into human readable answer. {"success": true, "total_detections": 120, "total_segments": 118} The crown segmentation tool successfully detected 120 trees and segmented 118 crowns in the uploaded image.


In [ ]:
save_dir = "/content/tree_tool_model"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("Model saved to:", save_dir)


Model saved to: /content/tree_tool_model


In [ ]:
# Train the model
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,020 | Num Epochs = 3 | Total steps = 384
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
25,0.772500
50,0.187000
75,0.183700
100,0.183300
125,0.182700
150,0.181100
175,0.181700
200,0.181400
225,0.182100
250,0.181700
